In [12]:
import pandas as pd
import re

from datetime import datetime

In [2]:
month_dict = {
    'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04',
    'mayo': '05', 'junio': '06', 'julio': '07', 'agosto': '08',
    'septiembre': '09', 'octubre': '10', 'noviembre': '11', 'diciembre': '12'
}

def interventions(text):
    pattern = r'([A-ZÁÉÍÓÚÑ][A-ZÁÉÍÓÚÑ\s]+?:\s*[\s\S]*?)(?=[A-ZÁÉÍÓÚÑ][A-ZÁÉÍÓÚÑ\s]+?:|\Z)'
    interventions = re.findall(pattern, text)
    return interventions


def president_interventio(text):
    match = 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR:'
    filtered_list = [elem for elem in text if elem.startswith(match)]
    return filtered_list


def convert_to_iso(date_str):
    parts = date_str.split()
    month = month_dict.get(parts[0].lower(), '01')
    day = parts[1].replace(',', '')
    year = parts[2]
    return f"{year}-{month}-{day.zfill(2)}"



## Discourses

In [ ]:
json_file = '../data/raw/discourse.json'

discourses = pd.read_json(json_file)

discourses['timestamp'] = discourses['entry-date'].apply(convert_to_iso)

discourses['interventions'] = discourses['entry-content'].apply(interventions)

discourses['president'] = discourses['interventions'].apply(president_interventio)


discourses = discourses[discourses['entry-title'].str.contains(
    'prensa matutina', case=False
)]

In [ ]:
discourses.sample(10)

In [5]:
discourses.drop(
    columns=['entry-date', 'entry-title']
).to_json('../data/prepro/discourses.json')

## Interventions

In [7]:
interventions = discourses.explode(
    'interventions', ignore_index=True
)
interventions['speaker'] = interventions['interventions'].astype(str).apply(
    lambda x: x.split(':', 1)[0] if len(x.split(':', 1)) > 1 else x)
interventions['speech'] = interventions['interventions'].astype(str).apply(
    lambda x: x.split(':', 1)[1] if len(x.split(':', 1)) > 1 else x
)
interventions.drop(
    columns=['interventions', 'entry-date', 'president', 'entry-title', 'entry-content', 'entry-url'], 
    inplace=True
)

In [ ]:
interventions.sample(10)

In [10]:
interventions.to_json('../data/prepro/interventions.json')